# CNN

In [1]:
!pip install wandb

In [2]:
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, LSTM
from keras.layers import Conv1D, Flatten, MaxPooling1D
from keras.datasets import imdb
import wandb
from wandb.keras import WandbCallback
import numpy as np
from keras.preprocessing import text
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline

In [3]:
df = pd.read_csv("/content/US-Economic-News.csv", delimiter=',', encoding= 'ISO-8859-1')

z = df['text']
o = df['headline']
X = o + z
y = df['relevance']
print(y.head())

y = pd.DataFrame(y, columns=['relevance'])

0    yes
1     no
2     no
3     no
4    yes
Name: relevance, dtype: object


In [4]:
categorical_columns = ['relevance']

# Define the transformation: OneHotEncoder for categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_columns)
    ],
    remainder='passthrough'  # This will pass through other columns not specified in transformers
)

# Create a pipeline with the preprocessor
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Fit and transform the DataFrame
y_ = pipeline.fit_transform(y)

# transformed_df is a NumPy array. To convert it back to a DataFrame:
y_ = pd.DataFrame(y_, columns=pipeline.named_steps['preprocessor'].get_feature_names_out())


In [5]:
y_ = y_.drop(columns=['cat__relevance_no', 'cat__relevance_not sure'])

In [6]:

# here stratify is used to balance dataset as much as possible
X_train, X_test, y_train, y_test = train_test_split(X, y_, test_size=0.2, random_state=42, stratify=y)


In [7]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')
import string

stop_words = set(stopwords.words('english'))
def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

X_train = X_train.apply(preprocess)
X_test = X_test.apply(preprocess)

In [8]:
#y_train =  pd.DataFrame.to_numpy(y_train)
y_train

,cat__relevance_yes
420,0.0
3350,0.0
6716,1.0
4224,0.0
6230,0.0
...,...
3923,0.0
876,0.0
2182,1.0
7333,0.0


In [9]:
X_train

420     trade deficit worry nearly halts activity euro...
3350    teaparty activists complicate republican comeb...
6716    elder shareone great feats social engineering ...
4224    article 1 ûó titlethe house banking coni mitt...
6230    trade shrinks selloffnew york sept 20 upûóthe...
                              ...                        
3923    wall street revives reward junkets top brokers...
876     uk inflation continues riselondonuk inflation ...
2182    gdp growth revised upward 3rd quarter slight c...
7333    remodeling longer adds reinvesting equity home...
2463    rich means staying waythe rich different f sco...
Length: 6400, dtype: object

In [10]:

wandb.init(
        project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "US-Economic-News.csv",
    "epochs": 10,
    }
)
config = wandb.config

# set parameters:
config.vocab_size = 1000
config.maxlen = 5000
config.batch_size = 32
config.embedding_dims = 10
config.filters = 16
config.kernel_size = 3
config.hidden_dims = 250
config.epochs = 10

tokenizer = text.Tokenizer(num_words=config.vocab_size)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_matrix(X_train)
X_test = tokenizer.texts_to_matrix(X_test)

X_train = sequence.pad_sequences(X_train, maxlen=config.maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=config.maxlen)


wandb: Currently logged in as: efeperro. Use `wandb login --relogin` to force relogin


In [11]:
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [12]:

model = Sequential()
model.add(Embedding(config.vocab_size,
                    config.embedding_dims,
                    input_length=config.maxlen))
model.add(Dropout(0.5))
model.add(Conv1D(config.filters,
                 config.kernel_size,
                 padding='valid',
                 activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(config.filters,
                 config.kernel_size,
                 padding='valid',
                 activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(config.hidden_dims, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=config.batch_size,
          epochs=config.epochs,
          validation_data=(X_test, y_test), callbacks=[WandbCallback()])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/10
200/200 [==============================] - ETA: 0s - loss: 0.4793 - accuracy: 0.8205

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20231112_194445-6uce6rjz/files/model-best)... Done. 0.3s


200/200 [==============================] - 49s 240ms/step - loss: 0.4793 - accuracy: 0.8205 - val_loss: 0.5299 - val_accuracy: 0.8225
Epoch 2/10
200/200 [==============================] - ETA: 0s - loss: 0.4359 - accuracy: 0.8225

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20231112_194445-6uce6rjz/files/model-best)... Done. 0.3s


...

200/200 [==============================] - 46s 229ms/step - loss: 0.4114 - accuracy: 0.8225 - val_loss: 0.4342 - val_accuracy: 0.8225
Epoch 5/10
200/200 [==============================] - 45s 223ms/step - loss: 0.4038 - accuracy: 0.8233 - val_loss: 0.4393 - val_accuracy: 0.8225
Epoch 6/10
200/200 [==============================] - ETA: 0s - loss: 0.4003 - accuracy: 0.8228

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20231112_194445-6uce6rjz/files/model-best)... Done. 0.3s


200/200 [==============================] - 46s 230ms/step - loss: 0.4003 - accuracy: 0.8228 - val_loss: 0.4280 - val_accuracy: 0.8238
Epoch 7/10
200/200 [==============================] - ETA: 0s - loss: 0.3863 - accuracy: 0.8247

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20231112_194445-6uce6rjz/files/model-best)... Done. 0.3s


200/200 [==============================] - 47s 233ms/step - loss: 0.3863 - accuracy: 0.8247 - val_loss: 0.4199 - val_accuracy: 0.8219
Epoch 8/10
200/200 [==============================] - 49s 243ms/step - loss: 0.3782 - accuracy: 0.8252 - val_loss: 0.4380 - val_accuracy: 0.8250
Epoch 9/10
200/200 [==============================] - 44s 221ms/step - loss: 0.3662 - accuracy: 0.8291 - val_loss: 0.4359 - val_accuracy: 0.8219
Epoch 10/10
200/200 [==============================] - 45s 227ms/step - loss: 0.3545 - accuracy: 0.8342 - val_loss: 0.4434 - val_accuracy: 0.8194


0.839 accuracy